# 𝕊𝕖𝕞𝕚-𝕊𝕦𝕡𝕖𝕣𝕧𝕚𝕤𝕖𝕕 𝕝𝕖𝕒𝕣𝕟𝕚𝕟𝕘 𝕦𝕤𝕚𝕟𝕘 𝕄𝕖𝕒𝕟 𝕋𝕖𝕒𝕔𝕙𝕖𝕣

Implementation of pixel-wise Mean Teacher (MT)
    
This method is proposed in the paper: 
    'Mean Teachers are Better Role Models:
        Weight-Averaged Consistency Targets Improve Semi-Supervised Deep Learning Results'
This implementation only supports Gaussian noise as input perturbation, and the two-heads
outputs trick is not available.

Source:
https://github.com/ZHKKKe/PixelSSL/blob/master/pixelssl/ssl_algorithm/ssl_mt.py


Todo:
* [] get everything on cuda, cpu -> to configs.device
* [] metrics for segmentation + unittests
* [] track the combined loss - what's the weight, ...


Things that we have to set to the device (gpu):
* the model
* the image that gets fed into the model
* the ground truth that gets fed into the criterion
* the ground truth that gets fed into the metrics - due to activation functions


recall high: most of the roi has been segmented
in combination with low precision: general oversegmentation

Using Learning Rate Scheduler:

https://discuss.pytorch.org/t/how-to-implement-torch-optim-lr-scheduler-cosineannealinglr/28797/5

```
steps = 10
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)

for epoch in range(5):
    for idx in range(steps):
        scheduler.step()
        print(scheduler.get_lr())

    print('Reset scheduler')
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)
```            


* TODO
* BINARY_MODE MULTICLASS_MODE - the loss wants a different encoding ...
* ground_truth = torch.nn.functional.one_hot(ground_truth, num_classes)  # N,H*W -> N,H*W, C
* ground_truth = ground_truth.permute(0, 2, 1)  # N, C, H*W
* RuntimeError: one_hot is only applicable to index tensor.

           
C:\Users\Christina\Documents\datasceyence\helper\dataset\template.py:38: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self.csv_data = pd.concat(csv_list, axis=0, ignore_index=False)
C:\Users\Christina\Documents\datasceyence\helper\dataset\template.py:38: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self.csv_data = pd.concat(csv_list, axis=0, ignore_index=False)
  
  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:355: error: (-215:Assertion failed) n >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::fitEllipseNoDirect'
  
  C:\Users\Christina\Documents\datasceyence\helper\compute\loss\shape.py:235: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(loss)
C:\Users\Christina\anaconda3\envs\chrisy\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
  
  
RuntimeError: stack expects each tensor to be equal size, but got [1, 128, 128] at entry 0 and [3, 128, 128] at entry 8
something in the dataloader is not grayscale


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.


# Imports

In [1]:
import glob
import os
import time
import random
from PIL import Image
import logging
import shutil # to remove dirs


import pandas as pd

import torch
import torch.nn
from torch.autograd import Variable
import torchvision
from torch.utils.tensorboard import SummaryWriter

import segmentation_models_pytorch as smp

from torch.utils.data import DataLoader

import cv2

import matplotlib.pyplot as plt

import sys
sys.path.insert(0, "helper")
from helper.dataset.meanteacher import *
# from helper.model.mean_teacher import * 
from helper.sampler.mixed_batch import *
# from helper.model.block.noise_block import GaussianNoiseBlock
from helper.compute.bin_seg import BCE_BinSeg_CU
from helper.compute.loss.shape import SignedDistanceLoss, HausdorffDTLoss
from helper.compute.loss.dice import DiceLoss
from helper.compute.metric.uncertainty import *
from helper.compute.metric.segmentation import *


#from pixelssl.utils import REGRESSION, CLASSIFICATION
#from pixelssl.utils import logger, cmd, tool
#from pixelssl.nn import func
#from pixelssl.nn.module import patch_replication_callback, GaussianNoiseLayer

C:\Users\Christina\anaconda3\envs\chrisy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.device_count()


2

# Experiment Configs

In [3]:
class Configs():
    
    def __init__(self, reset=False):
        
        
        # =============================================================================
        # CHECK THESE
        # =============================================================================
        reset_2 = True # double control, otherwise all results for this prefix will be deleted
        if reset and reset_2:
            reset=True
        else:
            reset=False
        self.prefix = "exp5_with_fda_try2" # put local if christina's computer
        self.reduced_data = False
        # "cuda" - use all the available GPUs
        # "cuda:1,3" - use specific GPUs            
        self.print_todos = False
        self.min_save_fscore = 0.7 # 0.7
        self.description = "with b0 model, dice loss for consistency and fda transform"
        # =============================================================================
        # CHECK THESE
        # =============================================================================
        
        if "local" in self.prefix:
            self.device = "cpu" # "cuda" if torch.cuda.is_available() else "cpu"
            self.base_path = r"C:/Users/Prinzessin/projects/decentnet"
            self.logger_path = f"C:/Users/Prinzessin/projects/decentnet/datasceyence/results/{self.prefix}/logs"
            self.save_checkpoint_path = f"C:/Users/Prinzessin/projects/decentnet/datasceyence/results/{self.prefix}/ckpts"
        else:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.base_path = r"C:/Users/Christina/Documents"
            self.logger_path = f"E:/Christina/datasceyence/results/{self.prefix}/logs"
            self.save_checkpoint_path = f"E:/Christina/datasceyence/results/{self.prefix}/ckpts"
        
        # smp unet ++ parameters
        self.encoder_name = "timm-efficientnet-b0" # "efficientnet-b7"
        self.encoder_weights = "imagenet"
        self.in_channels =  1
        self.n_output_neurons = 2
        
        self.eps = 1e-6
        
        self.num_workers = 4
        self.epochs = 200 # need more for final training ... more than 1000
        self.rampup_length = 50 # 600 for 1000
        
        # self.gaussian_noise = None # not in use
        
        self.ema_decay = 0.999 # default value
        
        # Sizes of tensors must match except in dimension 1
        # I solved the issue by resizing all the images size divisible to 32.
        self.image_size = 128 # 512
        
        # batch size = n_samples_per_class_per_batch * classes
        # for mixed batch sampling
        self.n_samples_per_class_per_batch = 10
        
        # self.lbs = 3 #  self.args.labeled_batch_size # .... remove this eventually and replace

        # optimisation
        # self.optimiser = "sgd"
        self.base_lr = 0.001 # 0.0001
        self.min_lr = 0.00001
        self.weight_decay = None #1e-4 # not in use
        self.momentum = 0.9
        
        # self.is_epoch_lrer = True # epoch or batch based learning rate updater
        
        # self.dropout = None
        
        
         
        # =============================================================================
        # Paths
        # =============================================================================
        
        
        if not os.path.exists(self.base_path):
            os.makedirs(self.base_path)
        os.chdir(self.base_path) # this is now the main directory !!!!!!!!!!!!!!!!!!!!
        
        self.csv_filenames = glob.glob(r"datasceyence/data_prep/mt_*.csv")
        self.csv_test_filenames = ["datasceyence/data_prep/mt_data_cirrus.csv"]
        
        print(self.csv_filenames)
        
        # input # should be none for actual experiments
        self.load_checkpoint_file = None # "E:/Christina/datasceyence/results/exp3_dice_cons_all_data_b7/ckpts/checkpoint_0.799_134.ckpt" 
        
        # all csv files used for run_mean_teacher.ipybn
        #self.csv_data_paths = [
        #    {"path" : r"data/data_ichallenge_amd.csv"}, 
        #    {"path" : r"data/data_ichallenge_non_amd.csv"}
        #]
        
        
        # output
        
        # if path exists: delete, create path
        if reset:
            if os.path.exists(self.logger_path):
                shutil.rmtree(self.logger_path)
            os.makedirs(self.logger_path)
            
        self.result_path = os.path.join(self.logger_path, f"metrics_{self.prefix}.csv" )
            
        
         # if path exists: delete, create path
        if reset:
            if os.path.exists(self.save_checkpoint_path):
                shutil.rmtree(self.save_checkpoint_path)
            os.makedirs(self.save_checkpoint_path)


    def log(self):
        # =============================================================================
        # save all class variables to file "configs.txt"
        # =============================================================================
        c = pd.DataFrame.from_dict({'key': self.__dict__.keys(), 'value': self.__dict__.values()})
        c.to_csv(os.path.join(self.logger_path, "configs.txt"), sep=':', index=False)

# Routine

In [4]:
class RoutineMT:

    def __init__(self, configs):
        super(RoutineMT, self).__init__()
        
        self.configs = configs
                
        self.load_ckpt = torch.load(configs.load_checkpoint_file) if configs.load_checkpoint_file is not None else None
        
        self.i_item_total = 0
        
        
        # =============================================================================
        # Logger
        # =============================================================================
        
        self.writer =  SummaryWriter(log_dir=self.configs.logger_path)
        logging.basicConfig(filename=os.path.join(self.configs.logger_path, 'logger.log'), encoding='utf-8', level=logging.INFO) # everything more important or INFO
        # https://docs.python.org/3/library/logging.html
        
        # =============================================================================
        # Models
        # =============================================================================
        s_model = smp.UnetPlusPlus(
                        encoder_name=self.configs.encoder_name,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
                        encoder_weights=self.configs.encoder_weights,  # use `imagenet` pre-trained weights for encoder initialization
                        in_channels=self.configs.in_channels,          # model input channels (1 for gray-scale images, 3 for RGB, etc.)
                        classes=self.configs.n_output_neurons,         # model output channels (number of classes in your dataset)
                    )
        
        t_model = smp.UnetPlusPlus(
                        encoder_name=self.configs.encoder_name,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
                        encoder_weights=self.configs.encoder_weights,  # use `imagenet` pre-trained weights for encoder initialization
                        in_channels=self.configs.in_channels,          # model input channels (1 for gray-scale images, 3 for RGB, etc.)
                        classes=self.configs.n_output_neurons,         # model output channels (number of classes in your dataset)
                    )
        
        # detach the teacher model
        for param in t_model.parameters():
            param.detach_()
            
        # model= nn.DataParallel(model)
        # if only one gpu ??
        if "cpu" in self.configs.device:
            self.models = {'s': s_model, 
                           't': t_model}
        else: 
            self.models = {'s': torch.nn.DataParallel(s_model).to(self.configs.device), 
                           't': torch.nn.DataParallel(t_model).to(self.configs.device)}
        
        # add gaussian noise
        # currently not in use
        # self.gaussian_noiser = GaussianNoiseBlock(self.configs.gaussian_noise).cuda()
        
        # =============================================================================
        # Computing Units
        # =============================================================================
        self.computing_unit = {
            "s_train" : BCE_BinSeg_CU(n_output_neurons=self.configs.n_output_neurons, mode="train", model="s", device=self.configs.device, writer=self.writer),
            "s_val"   : BCE_BinSeg_CU(n_output_neurons=self.configs.n_output_neurons, mode="val", model="s", device=self.configs.device, writer=self.writer),
            "t_train" : BCE_BinSeg_CU(n_output_neurons=self.configs.n_output_neurons, mode="train", model="t", device=self.configs.device, writer=self.writer),
            "s_test"  : BCE_BinSeg_CU(n_output_neurons=self.configs.n_output_neurons, mode="val", model="s", device=self.configs.device, writer=None),
        }
        
        self.top_val = { #'lowest_loss' : np.inf,  # low good
                         'highest_fscore' : self.configs.min_save_fscore, # high good
                         #'highest_jac' : 0, # high good
                         #'highest_symhd' : np.inf # low good
                     }
        
        
        
        # =============================================================================
        # Loss functions
        # =============================================================================
        # TODO: support more types of the consistency criterion
        # something with head and each head has a loss function attached??
        self.criterions = {'shape': HausdorffDTLoss(), # (device=self.configs.device, n_output_neurons=self.configs.n_output_neurons), # criterion_funcs[0](self.args),
                           'pixel': DiceLoss(n_output_neurons=self.configs.n_output_neurons, softmax=True), # added softmax on 29-may-2023 4pm
                           'cons': DiceLoss(n_output_neurons=self.configs.n_output_neurons, softmax=True) # ,torch.nn.MSELoss() # softmax missing? there is a softmax mse loss ?? todo
                          }
        
        self.um = UncertaintyMetric(n_noise=4, n_repeat=2, n_output_neurons=self.configs.n_output_neurons)
        
        # =============================================================================
        # Datasets: train, val, test
        # =============================================================================
                
        train_set = MeanTeacherTrainDataset(mode="train", channels=self.configs.in_channels, image_size=self.configs.image_size, csv_filenames=self.configs.csv_filenames, reduced_data=self.configs.reduced_data)
        train_mbs = MixedBatchSampler(train_set.get_mbs_labels(), n_samples_per_class_per_batch=self.configs.n_samples_per_class_per_batch)
        
        print("*"*50)
        print("all labels", train_set.get_mbs_labels())
        
        x = train_set.get_mbs_labels()
        leastFrequent = min(x, key=x.count)
        amount = x.count(leastFrequent)
        print("this needs some error catching")
        self.steps = int(amount/self.configs.n_samples_per_class_per_batch)
        print("steps", self.steps)
                
        val_set = MeanTeacherValDataset(mode="val", channels=self.configs.in_channels, image_size=self.configs.image_size, csv_filenames=self.configs.csv_filenames, reduced_data=self.configs.reduced_data)
        
        cir_set = MeanTeacherCirDataset(channels=self.configs.in_channels, csv_filenames=self.configs.csv_test_filenames, image_size=self.configs.image_size)
              
        self.dataloader = {"train" : DataLoader(train_set, batch_sampler=train_mbs),
                           "val"   : DataLoader(val_set),
                           "cir"   : DataLoader(cir_set, shuffle=False)
                          }
        
        
        # =============================================================================
        # Optimisers
        # =============================================================================
        self.optims = {'s': torch.optim.SGD(self.models["s"].parameters(), lr=self.configs.base_lr, momentum=self.configs.momentum) # optimizer_funcs[0](self.models["s"].module.param_groups)
                          }

        # =============================================================================
        # Learning rate schedulers
        # =============================================================================
        self.lrsers = {'s': torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(self.optims["s"], 
                                                                              T_0 = 100, # number of iterations for the first restart.
                                                                              eta_min = self.configs.min_lr
                                                                               )
                     }
                  
        # =============================================================================
        # Resume training
        # =============================================================================
        if self.load_ckpt:
            self.models["s"].load_state_dict(self.load_ckpt['s_model'])
            self.models["t"].load_state_dict(self.load_ckpt['t_model'])
            self.optims["s"].load_state_dict(self.load_ckpt['s_optim'])
            self.lrsers["s"].load_state_dict(self.load_ckpt['s_lrer'])
            try: 
                self.start_epoch = self.load_ckpt['epoch']
            except:
                self.start_epoch = self.load_ckpt['i_epoch']
        else:
            self.start_epoch = 0
            

    
    def run_training(self, i_epoch):
        # =============================================================================
        # Training
        # =============================================================================
        
        mode="train"
        
        self.models["s"].train()
        self.models["t"].eval() # train or eval ... ???
        
        for i_item, item in enumerate(self.dataloader[mode]):
            # =============================================================================
            # Process Batch
            # =============================================================================
            
            # unlabelled for consistency loss
            unlabelled_batch_ids = np.where(item["has_mask"] == False) 
            # labelled for task loss
            labelled_batch_ids = np.where(item["has_mask"] == True) 
            
            # get predictions of student model for all images
            s_model_output = self.models["s"](item["img"].to(self.configs.device)) # we want all images (task loss and consistency loss)
            
            # run batch for student model
            self.computing_unit["s_train"].run_batch(configs=self.configs, criterions=self.criterions, model_output=s_model_output[labelled_batch_ids], ground_truth=item["msk"][labelled_batch_ids])
            
            if False:
                print(labelled_batch_ids)
                print(unlabelled_batch_ids)
                print("item"*40)
                print(item)
                print("item"*40)
                print("s model")
                print(s_model_output)
                print(item["msk"])
                print(s_model_output.shape)
                print(item["msk"].shape)
                print("next")

            # =============================================================================
            # Teacher Model
            # =============================================================================
            
            no_grad_img = Variable(item["img"], requires_grad=False)
            
            # forward the teacher model
            with torch.no_grad():
                # get predictions of teacher model for all images
                t_model_output = self.models["t"](no_grad_img.to(self.configs.device)) # we want all images (task loss and consistency loss)               
                self.computing_unit["t_train"].run_batch(configs=self.configs, criterions=self.criterions, model_output=t_model_output[labelled_batch_ids], ground_truth=item["msk"][labelled_batch_ids])
                            
            if self.configs.print_todos: 
                print("run_meanteacher todo: the uncertainty mask is not good - for consistency loss")    
            # uncertainy_mask = self.um.run(self.models["t"], item["img"].to(self.configs.device))
            
            # =============================================================================
            # Consistency Loss
            # =============================================================================
            
            def sigmoid_rampup(current, rampup_length):
                # calculate the ramp-up coefficient of the consistency constraint
                # returns vavlue between 0 and 1
                # https://github.com/HiLab-git/SSL4MIS/blob/master/code/utils/ramps.py
                # Consistency ramp-up from https://arxiv.org/abs/1610.02242
                if rampup_length == 0:
                    return 1.0
                else:
                    current = np.clip(current, 0.0, rampup_length)
                    phase = 1.0 - current / rampup_length
                    return float(np.exp(-5.0 * phase * phase))
                        
            cons_weight = sigmoid_rampup(i_epoch, self.configs.rampup_length)
            
            
            if self.writer:
                self.writer.add_scalars("info/weight", {"cons_ramp" : cons_weight}, self.i_item_total)
            
            # calculate the consistency constraint from the teacher model to the student model
            if self.configs.print_todos: 
                print("run_meanteacher todo: here inside we have to do some beautifying of the mask - for consistency loss")
            # https://discuss.pytorch.org/t/how-does-applying-a-mask-to-the-output-affect-the-gradients/126520
            
            def create_circle(model_output):
                
                
                # 3 dims (batch, widht, height)
                _, highest_class = torch.max(model_output, 1)  
                
                # 3 dims (batch, widht, height)
                final_masks = torch.zeros_like(highest_class)
                
                #print(highest_class.shape)
                
                for i, output in enumerate(highest_class):
                    
                    output = output.detach().cpu().numpy()
                    
                    #plt.figure()
                    #plt.imshow(output)
                    
                    max_area = 0
                    circle = np.zeros(output.shape, dtype = np.uint8)
                    
                    num_labels, labels_im = cv2.findContours(output, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)    
                    if len(num_labels) > 0:
                        max_area = max(num_labels, key = cv2.contourArea)
                    
                        try:
                            #ellipse = cv2.fitEllipse(max_area)

                            #cv2.ellipse(circle, ellipse, (1), thickness=-1)

                            cv2.drawContours(circle, [max_area], -1, color=(1), thickness=-1)

                            final_masks[i] = torch.tensor(circle)

                            #plt.figure()
                            #plt.imshow(circle)
                        except Exception as e:
                            # logging.info("fitting the ellipse didn't work")
                            logging.info("max area %d", cv2.contourArea(max_area))
                            logging.info(e)
                            print("probably empty mask", cv2.contourArea(max_area))
                            #final_masks[i, 0] = model_output[i, 0]
                            final_masks[i] = highest_class[i]
                    else:
                        
                        print(final_masks.shape)
                        print(highest_class.shape)
                        print(final_masks[i].shape)
                        print(highest_class[i].shape)
                        
                        # 2 dims + 1 = 3 dims
                        final_masks[i] = highest_class[i]
                
                if self.configs.print_todos: 
                    print(final_masks.shape)
                
                # 3 dims (batch, 1, widht, height)
                final_masks = final_masks.unsqueeze(1)
                
                if self.configs.print_todos: 
                    print(final_masks.shape)
                
                return final_masks # Variable(final_masks, requires_grad=False) # circle
            
            circle = create_circle(t_model_output[unlabelled_batch_ids])

            t_pseudo_gt = circle.to(self.configs.device) # t_model_output * circle # Variable(t_model_output.detach().data, requires_grad=False) # t_model_output
            
            if self.configs.print_todos: 
                print("modeeeeeeeeeeeeeel output of teeeacheeer")
                print(t_pseudo_gt)
                print(t_pseudo_gt.shape)
                
                
                print(" hier ist ein subplot")

                fig, axs = plt.subplots(1, 3)
                _, highest_class = torch.max(s_model_output[unlabelled_batch_ids], 1)  
                highest_class = highest_class[0].detach().cpu().numpy()
                axs[0].imshow(highest_class, cmap="cool")
                axs[0].axis('off')
                axs[1].imshow(t_pseudo_gt[0,0].detach().cpu().numpy(), cmap="cool")
                axs[1].axis('off')


                _, highest_class2 = torch.max(t_model_output[unlabelled_batch_ids], 1)  
                highest_class2 = highest_class2[0].detach().cpu().numpy()
                axs[2].imshow(highest_class2, cmap="cool")
                axs[2].axis('off')
                fig.show()
                
            

            
            if unlabelled_batch_ids:
                cons_loss = self.criterions["cons"](s_model_output[unlabelled_batch_ids], t_pseudo_gt)
            else:
                cons_loss = self.zero_tensor
                            
            # this is a float
            threshold = torch.tensor( ( 0.75 + (0.25*sigmoid_rampup(i_epoch, self.configs.rampup_length)) * np.log(2) ) ).to(self.configs.device)
            
            
            if self.writer:
                self.writer.add_scalars("info/weight", {"thresh_ramp" : threshold.detach().cpu().numpy()}, self.i_item_total)
            
            #print(uncertainy_mask)
            #print(threshold)
            
            # this is not on the gpu ... now it is ...
            #bin_uncertainy_mask = (uncertainy_mask.to(self.configs.device) < threshold) # .float()
            
            #cons_loss_masked = torch.sum(bin_uncertainy_mask * cons_loss) / (2*torch.sum(bin_uncertainy_mask)+self.configs.eps)

            # =============================================================================
            # Backprop for student model
            # =============================================================================
            
            # combined loss
            # supervised loss + consistency loss * consistency weight
            loss = self.computing_unit["s_train"].task_loss + cons_loss * cons_weight #  + cons_loss_masked * cons_weight
            
            self.writer.add_scalars("info/loss", {"backprop" : loss,
                                                  "cons"  : cons_loss,
                                                  "cons_weighted"  : cons_loss * cons_weight,
                                                  "shape" : self.computing_unit["s_train"].shape_loss,
                                                  "pixel" : self.computing_unit["s_train"].pixel_loss}, self.i_item_total)
            
            self.optims["s"].zero_grad()
            loss.backward()
            self.optims["s"].step()      
            
            # =============================================================================
            # LR Scheduler = every batch a step
            # =============================================================================
            self.lrsers["s"].step()
            if self.writer:
                self.writer.add_scalars("info/lr", {"s_train" : self.lrsers["s"].get_last_lr()[0]}, self.i_item_total)
            
            # =============================================================================
            # EMA for teacher model
            # https://github.com/HiLab-git/SSL4MIS/blob/master/code/train_uncertainty_aware_mean_teacher_2D.py
            # https://github.com/ZHKKKe/PixelSSL/blob/master/pixelssl/ssl_algorithm/ssl_mt.py
            # =============================================================================
            self.models["t"].train() # this should be train for ema
            
            local_ema_decay = min(1 - 1 / (self.i_item_total + 1), self.configs.ema_decay)
            for t_param, s_param in zip(self.models["t"].parameters(), self.models["s"].parameters()):
                # model_weights = decay * model_weights + (1 - decay) * new_model_weights
                t_param.data = t_param.data * local_ema_decay +  s_param.data * (1 - local_ema_decay)
        
            self.i_item_total += 1
        
        # =============================================================================
        # Epoch Process (basically logging)
        # =============================================================================
        self.computing_unit["s_train"].run_epoch(i_epoch=i_epoch)
        self.computing_unit["t_train"].run_epoch(i_epoch=i_epoch)
        
        

        if self.configs.print_todos: 
            print("lr")
            print(self.lrsers["s"].get_last_lr())
            print(self.lrsers["s"].get_last_lr()[0])
        
        # =============================================================================
        # Epoch log and reset (training)
        # =============================================================================
        
        if self.configs.print_todos: 
            print("training task loss")
            print(self.computing_unit["s_train"].epoch_collector["taskloss"])

            print("training fscore")
            print(self.computing_unit["s_train"].epoch_collector["fscore"])
        
        self.computing_unit["s_train"].log(csv_file_path = self.configs.result_path)
        self.computing_unit["t_train"].log(csv_file_path = self.configs.result_path)
        self.computing_unit["s_train"].reset_epoch()
        self.computing_unit["t_train"].reset_epoch()
        
    def run_validation(self, i_epoch):
        self.models["s"].eval()
        
        # =============================================================================
        # for each batch
        # =============================================================================

        for i_item, item in enumerate(self.dataloader["val"]):
            # =============================================================================
            # Student
            # =============================================================================

            with torch.no_grad():
                # get predictions of student model for all images
                s_model_output = self.models["s"](item["img"].to(self.configs.device)) # we want all images (task loss and consistency loss)
                
                # should be based on loss function!!!!! - can we do all of them in the loss function? this is annoying
                # s_model_output = torch.nn.functional.softmax(s_model_output, dim=1) # I DON'T KNOOOOW

                # run batch for student model
                self.computing_unit["s_val"].run_batch(configs=self.configs, criterions=self.criterions, model_output=s_model_output, ground_truth=item["msk"])
        
        
        # =============================================================================
        # Epoch Process (basically logging)
        # =============================================================================
        self.computing_unit["s_val"].run_epoch(i_epoch=i_epoch)

        # =============================================================================
        # Epoch log and reset (validation)
        # =============================================================================
        
        if self.configs.print_todos: 
            print("validation task loss")
            print(self.computing_unit["s_val"].epoch_collector["taskloss"])

            print("validation fscore")
            print(self.computing_unit["s_val"].epoch_collector["fscore"])
        
        self.computing_unit["s_val"].log(csv_file_path = self.configs.result_path)
        self.computing_unit["s_val"].reset_epoch()
        
        logging.info("Validation of epoch %d done" % i_epoch)
        
    def run_final_evaluation(self, i_epoch):
                
        model = self.models["s"]
        
        paths = glob.glob(os.path.join(self.configs.save_checkpoint_path, f'checkpoint_*_{i_epoch}.ckpt'))
        
        model.load_state_dict(torch.load(paths[0])['s_model'])
        model.eval()

        for i_item, item in enumerate(self.dataloader["cir"]):

            with torch.no_grad():
                # get predictions of student model for all images
                s_model_output = self.models["s"](item["img"].to(self.configs.device)) # we want all images (task loss and consistency loss)
                
                # should be based on loss function!!!!! - can we do all of them in the loss function? this is annoying
                # s_model_output = torch.nn.functional.softmax(s_model_output, dim=1) # I DON'T KNOOOOW

                # run batch for student model
                self.computing_unit["s_test"].run_batch(configs=self.configs, criterions=self.criterions, model_output=s_model_output, ground_truth=item["msk"])
                
                print(s_model_output.shape)
                
                fig, axs = plt.subplots(1, 4)
                axs[0].imshow(item["img"][0].permute(1, 2, 0))
                _, highest_class = torch.max(s_model_output, 1)  
                print(highest_class.shape)
                highest_class = highest_class[0].detach().cpu().numpy()
                print(highest_class.shape)
                axs[1].imshow(highest_class, cmap="cool")
                axs[1].axis('off')
                from skimage import measure
                contours = measure.find_contours(highest_class, 0.5)
                axs[2].imshow(item["img"][0].permute(1, 2, 0), cmap="gray")
                for contour in contours:
                    axs[2].plot(contour[:, 1], contour[:, 0], linewidth=2, color = 'mediumvioletred')
                axs[2].axis('off')
                axs[3].imshow(item["msk"][0].squeeze())
                axs[3].axis('off')
                fig.savefig("artefact.jpg", dpi=1000)

                
                
                
            if i_item == 10:
                break
        
        
        # =============================================================================
        # Epoch Process (basically logging)
        # =============================================================================
        self.computing_unit["s_test"].run_epoch(i_epoch=i_epoch)

        # =============================================================================
        # Epoch log and reset (validation)
        # =============================================================================
        
        print("Test task loss")
        print(self.computing_unit["s_test"].epoch_collector["taskloss"])
        
        print("Test fscore")
        print(self.computing_unit["s_test"].epoch_collector["fscore"])
        
        
        
    def log(self, i_epoch):
        # =============================================================================
        # Save checkpoint
        # =============================================================================
                
        # this cannot woooooooooooooork
        # we save the best already in the epoch .... ?
        
        #print("black magic " * 10)
        #print(self.top_val['highest_fscore'])
        #print(self.computing_unit["s_val"].top_collector["highest_fscore"] )
        
        if i_epoch > 2: #  and self.computing_unit["s_val"].epoch_collector["fscore"] > self.computing_unit["s_val"].top["highest_fscore"]:
            

            
            if self.top_val['highest_fscore'] < self.computing_unit["s_val"].top_collector["highest_fscore"]:
                
                self.top_val['highest_fscore'] = self.computing_unit["s_val"].top_collector["highest_fscore"] 
                
                #print(self.top_val['highest_fscore'])
            
                checkpoint = {
                    'name': self.configs.prefix,
                    'i_epoch': i_epoch, 
                    's_model': self.models["s"].state_dict(),
                    't_model': self.models["t"].state_dict(),
                    's_optim': self.optims["s"].state_dict(),
                    's_lrer': self.lrsers["s"].state_dict()
                }
                
                # self.computing_unit["s_val"].epoch_collector["fscore"]

                tmp = self.top_val['highest_fscore']
                checkpoint_path = os.path.join(self.configs.save_checkpoint_path, f'checkpoint_{tmp}_{i_epoch}.ckpt')
                torch.save(checkpoint, checkpoint_path)

                # self.computing_unit["s_val"].top["fscore"] = self.computing_unit["s_val"].epoch_collector["fscore"]

                logging.info("Saved model at epoch " + str(i_epoch) + " with fscore " + str(self.top_val['highest_fscore']))
            
        
        




# Run

In [5]:

# Configs
configs = Configs(reset=True)
configs.log()

# Run
run = RoutineMT(configs)

for i_epoch in range(run.start_epoch+1, configs.epochs):

    run.run_training(i_epoch=i_epoch)
    run.run_validation(i_epoch=i_epoch)
    run.log(i_epoch=i_epoch)

    if i_epoch > 1 and configs.reduced_data:
        break



['datasceyence/data_prep\\mt_data_cirrus.csv', 'datasceyence/data_prep\\mt_data_ichallenge_amd.csv', 'datasceyence/data_prep\\mt_data_plex.csv']


C:\Users\Christina\Documents\datasceyence\helper\dataset\template.py:39: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self.csv_data = pd.concat(csv_list, axis=0, ignore_index=False)
C:\Users\Christina\Documents\datasceyence\helper\dataset\template.py:39: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self.csv_data = pd.concat(csv_list, axis=0, ignore_index=False)


**************************************************
all labels [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\Users\Christina\Documents\datasceyence\helper\compute\loss\shape.py:235: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(loss)
C:\Users\Christina\anaconda3\envs\chrisy\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


{'name': 'BCE_BinSeg_CU', 'model': 's', 'mode': 'train', 'epoch': 1, 'datasize': 540, 'taskloss': 0.569, 'acc': 0.025, 'fscore': 0.047, 'jac': 0.024, 'prec': 0.025, 'rec': 0.358, 'symhd': 10.907}
{'name': 'BCE_BinSeg_CU', 'model': 't', 'mode': 'train', 'epoch': 1, 'datasize': 540, 'taskloss': 0.6, 'acc': 0.025, 'fscore': 0.046, 'jac': 0.024, 'prec': 0.025, 'rec': 0.416, 'symhd': 11.097}
{'name': 'BCE_BinSeg_CU', 'model': 's', 'mode': 'val', 'epoch': 1, 'datasize': 147, 'taskloss': 0.579, 'acc': 0.038, 'fscore': 0.07, 'jac': 0.037, 'prec': 0.038, 'rec': 0.523, 'symhd': 10.829}
{'name': 'BCE_BinSeg_CU', 'model': 's', 'mode': 'train', 'epoch': 2, 'datasize': 540, 'taskloss': 0.489, 'acc': 0.045, 'fscore': 0.076, 'jac': 0.04, 'prec': 0.045, 'rec': 0.277, 'symhd': 9.461}
{'name': 'BCE_BinSeg_CU', 'model': 't', 'mode': 'train', 'epoch': 2, 'datasize': 540, 'taskloss': 0.542, 'acc': 0.03, 'fscore': 0.055, 'jac': 0.028, 'prec': 0.03, 'rec': 0.357, 'symhd': 10.713}
{'name': 'BCE_BinSeg_CU', 'mo

C:\Users\Christina\Documents\datasceyence\helper\compute\metric\segmentation.py:105: RuntimeWarning: divide by zero encountered in double_scalars
  pixel_acc = (np.sum(matrix[0, :]) + self.eps) / (np.sum(matrix[0, :]) + np.sum(matrix[1, :]))


{'name': 'BCE_BinSeg_CU', 'model': 's', 'mode': 'val', 'epoch': 16, 'datasize': 147, 'taskloss': 0.184, 'acc': inf, 'fscore': 0.605, 'jac': 0.474, 'prec': 0.688, 'rec': 0.593, 'symhd': inf}
{'name': 'BCE_BinSeg_CU', 'model': 's', 'mode': 'train', 'epoch': 17, 'datasize': 540, 'taskloss': 0.176, 'acc': 0.593, 'fscore': 0.617, 'jac': 0.449, 'prec': 0.593, 'rec': 0.649, 'symhd': 5.119}
{'name': 'BCE_BinSeg_CU', 'model': 't', 'mode': 'train', 'epoch': 17, 'datasize': 540, 'taskloss': 0.275, 'acc': 0.449, 'fscore': 0.476, 'jac': 0.314, 'prec': 0.449, 'rec': 0.515, 'symhd': 5.076}
{'name': 'BCE_BinSeg_CU', 'model': 's', 'mode': 'val', 'epoch': 17, 'datasize': 147, 'taskloss': 0.172, 'acc': 0.62, 'fscore': 0.634, 'jac': 0.496, 'prec': 0.62, 'rec': 0.701, 'symhd': 5.605}
{'name': 'BCE_BinSeg_CU', 'model': 's', 'mode': 'train', 'epoch': 18, 'datasize': 540, 'taskloss': 0.166, 'acc': 0.612, 'fscore': 0.637, 'jac': 0.468, 'prec': 0.612, 'rec': 0.669, 'symhd': 5.133}
{'name': 'BCE_BinSeg_CU', 'mod

KeyboardInterrupt: 

In [ ]:
# Configs
configs = Configs(reset=False)
configs.log()

# Run
run = RoutineMT(configs)

i_epoch = 50

run.run_final_evaluation(i_epoch=i_epoch)


In [ ]:
# Configs
configs = Configs(reset=False)
configs.log()

# Run
run = RoutineMT(configs)

i_epoch = 98

run.run_final_evaluation(i_epoch=i_epoch)


In [6]:
# tensorboard --logdir projects\decentnet\results\exp1_mt\logs port=6005   
# http://localhost:6005/?runColorGroup=regex%3A(_s_train|_s_val|_t_)#timeseries

# regex: (_s_train|_s_val|_t_)


%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir E:/Christina/datasceyence/results/exp5_with_fda_try2/logs --port 6043

In [ ]:
%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir E:/Christina/datasceyence/results/exp4_max_mask_b0/logs --port 6041